In [1]:
import os

os.environ['NVIDIA_API_KEY'] = "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"
from omegaconf import OmegaConf
import sys
sys.path.append("../")
sys.path.append("../../../")
from retriever_evalset_generator import RetrieverEvalSetGenerator
from filters import EasinessFilter, AnswerabilityFilter
from nemo_curator.modules.filter import ScoreFilter, Score
from nemo_curator.datasets import DocumentDataset
import warnings
warnings.filterwarnings('ignore')


/home/vinay/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
import pandas as pd
df = pd.read_json("../data/nq_test.jsonl", lines=True)
df = df[:5]

In [3]:
cfg = OmegaConf.load("../config/retriever_eval_sdg.yaml")
retrieval_evalset_generator = RetrieverEvalSetGenerator(cfg)

In [4]:
dataset = DocumentDataset.from_pandas(df)
generated_dataset = retrieval_evalset_generator(dataset)

In [5]:
generated_dataset.df.head(n=5, npartitions=-1)

,_id,text,title,question,answer
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),Who are the executive producer and producers o...,The executive producer of the fourth season of...
1,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),When did the fourth season of Chicago Fire pre...,The fourth season of Chicago Fire premiered on...
2,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),How many episodes were there in the fourth sea...,There were 23 episodes in the fourth season of...
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C..."
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ..."


In [6]:
ef = EasinessFilter(cfg)
easiness_filter = ScoreFilter(ef,
                              text_field = ["text", "question"],
                              score_field = "easiness_scores")
af = AnswerabilityFilter(cfg)
answerability_filter = ScoreFilter(af,
                              text_field = ["text", "question"],
                              score_field = "answerability_scores")

In [7]:
filtered_dataset = easiness_filter(generated_dataset)
filtered_dataset.df.head(n=5, npartitions=-1)

,_id,text,title,question,answer,easiness_scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),Who are the executive producer and producers o...,The executive producer of the fourth season of...,0.621422
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",0.583219
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",0.632935
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,"Who wrote the song ""Fishin' in the Dark""?","""Fishin' in the Dark"" was written by Wendy Wal...",0.645659
8,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,"When was ""Fishin' in the Dark"" released as a s...","""Fishin' in the Dark"" was released as a single...",0.636784


In [8]:
filtered_dataset_2 = answerability_filter(filtered_dataset)
filtered_dataset_2.df.head(n=5, npartitions=-1)

Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series
Parse error the JSON object must be str, bytes or bytearray, not Series


,_id,text,title,question,answer,easiness_scores,answerability_scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),Who are the executive producer and producers o...,The executive producer of the fourth season of...,0.621422,"{\n""criterion_1_explanation"": ""The question is..."
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",0.583219,"{\n""criterion_1_explanation"": ""The question is..."
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",0.632935,"{\n""criterion_1_explanation"": ""The question is..."
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,"Who wrote the song ""Fishin' in the Dark""?","""Fishin' in the Dark"" was written by Wendy Wal...",0.645659,"{\n""criterion_1_explanation"": ""The question is..."
7,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,"Which American country music group recorded ""F...","""Fishin' in the Dark"" was recorded by American...",0.660160,"{\n""criterion_1_explanation"": ""The question is..."
